# Vectorizer + NaiveBayes Tuning

🎯 The goal of this challenge is to create a Pipeline combining a Vectorizer + a NaiveBayes algorithm and to fine-tune the pipeline.

✍️ Let's reuse the previous dataset with $2000$ reviews classified either as "positive" or "negative".

In [1]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data["target_encoded"] =  le.fit_transform(data.target)

In [3]:
data.head()

,target,reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...",0
1,neg,the happy bastard's quick movie review \ndamn ...,0
2,neg,it is movies like these that make a jaded movi...,0
3,neg,""" quest for camelot "" is warner bros . ' firs...",0
4,neg,synopsis : a mentally unstable man undergoing ...,0


## Preprocessing

❓ **Question (Cleaning)** ❓

Clean your texts

In [4]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    sentence= sentence.lower()
    sentence= ''.join(w for w in sentence if not w.isdigit())
    for punc in string.punctuation:
        sentence= sentence.replace(punc, ' ')
    sentence= sentence.strip()
    token= word_tokenize(sentence)
    clean_review= ''.join(WordNetLemmatizer().lemmatize(token) for token in sentence)
    return clean_review

In [5]:
# Clean reviews
data['cleaned_reviews']= data['reviews'].apply(preprocessing)

In [6]:
data.head()

,target,reviews,target_encoded,cleaned_reviews
0,neg,"plot : two teen couples go to a church party ,...",0,plot two teen couples go to a church party ...
1,neg,the happy bastard's quick movie review \ndamn ...,0,the happy bastard s quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...,0,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs...",0,quest for camelot is warner bros first f...
4,neg,synopsis : a mentally unstable man undergoing ...,0,synopsis a mentally unstable man undergoing ...


## Tuning

❓ **Question (Pipelining a Vectorizer and a NaiveBayes Model)** ❓

* Create a Pipeline that chains a vectorizer of your choice with a NaiveBayes model
* Optimize it
* What is your best estimator ?

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import set_config; set_config("diagram")

# Create Pipeline
pipeline= make_pipeline(TfidfVectorizer(), MultinomialNB())
# pipeline.get_params()

# Set parameters to search
params= {'tfidfvectorizer__ngram_range':((1,1),(2,2),(1,2),(2,1)), 'multinomialnb__alpha':(0.1, 1,2)}

# Perform grid search on pipeline
grid_search= GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=5, scoring='accuracy')

In [37]:
grid_search.fit(data['cleaned_reviews'], data["target_encoded"])

/home/parissa/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/parissa/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/parissa/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/parissa/.pyenv/versions/3.10.6/envs/lewagon/lib

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                                       ('multinomialnb', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'multinomialnb__alpha': (0.1, 1, 2),
                         'tfidfvectorizer__ngram_range': ((1, 1), (2, 2),
                                                          (1, 2), (2, 1))},
             scoring='accuracy')

In [38]:
grid_search.best_params_

{'multinomialnb__alpha': 0.1, 'tfidfvectorizer__ngram_range': (1, 2)}

In [39]:
grid_search.best_score_

0.8390000000000001

In [40]:
best_model= grid_search.best_estimator_

In [41]:
best_model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('multinomialnb', MultinomialNB(alpha=0.1))])

🏁 Congratulations! You've managed to chain a Vectorizer and a NLP model and fine-tuned it!

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!